In [0]:
!apt-get install p7zip-full

!unzip /content/test_numeric.csv.zip -d /content/data
!unzip /content/test_date.csv.zip -d /content/data
!unzip /content/test_categorical.csv.zip -d /content/data
!unzip /content/train_numeric.csv.zip -d /content/data
!unzip /content/train_date.csv.zip -d /content/data
!unzip /content/train_categorical.csv.zip -d /content/data

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ntu-qbs-assignment-2
!ls

In [0]:
from zipfile import ZipFile
import pandas as pd
import numpy as np # linear algebra

# Date data

In [0]:
train_date = pd.read_csv('/content/data/train_date.csv').astype('float32')
test_date = pd.read_csv('/content/data/test_date.csv').astype('float32')

In [0]:
train_date = train_date.drop("Id", axis=1)
test_date = test_date.drop("Id", axis=1)

# Numerical data

In [0]:
test_numeric = pd.read_csv('/content/data/test_numeric.csv').astype('float32')
train_numeric = pd.read_csv('/content/data/train_numeric.csv').astype('float32')

In [0]:
train_numeric = train_numeric.drop("Id", axis=1)
test_numeric = test_numeric.drop("Id", axis=1)

# Reduce momerry

In [0]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float16)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")

        else: #string case

            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)

            props[col] = props[col].astype('category')

             # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")


    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")

    return props

# Categorical data

In [0]:
import  numpy as np

data = pd.read_csv("/content/data/train_categorical.csv",chunksize=1)

for df in data:
    column_name = df.columns
    break

train_categorical = pd.read_csv("/content/data/train_categorical.csv",usecols=column_name[0:141])[column_name[0:141]]
train_categorical = reduce_mem_usage(train_categorical)

for i in range(0,2000,400):
    temp = pd.read_csv("/content/data/train_categorical.csv", usecols=column_name[(141+i):(541+i)])[column_name[(141+i):(541+i)]]
    train_categorical = pd.concat([train_categorical, reduce_mem_usage(temp)], axis=1, ignore_index=True)


test_categorical = pd.read_csv("/content/data/test_categorical.csv",usecols=column_name[0:141])[column_name[0:141]]
test_categorical = reduce_mem_usage(test_categorical)

for i in range(0,2000,400):
    temp = pd.read_csv("/content/data/test_categorical.csv", usecols=column_name[(141+i):(541+i)])[column_name[(141+i):(541+i)]]
    test_categorical = pd.concat([test_categorical, reduce_mem_usage(temp)], axis=1, ignore_index=True)

train_categorical.columns = column_name

In [0]:
train_categorical.isnull().sum()